# AI Agents in LangGraph – Lesson 5 

This notebook builds on the agents we created in **Lesson 4 (Persistence and Streaming )**.  
The key additions are:

* **Custom `reduce_messages`** – lets us *replace* earlier messages that share an `id` instead of always appending (the default `operator.add` behaviour you saw last time).  
* **`interrupt before "action"`** – inserts a breakpoint **before** the `action` node executes, so a human can inspect/modify the state or even *veto* the tool call.
* **`time travel`** – allows a human to *undo* previous actions and try a different path.



In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from uuid import uuid4

from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

### 1. Replacing messages instead of just appending

In [2]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In Leeson 4 just annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""

def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    """Merge `right` into `left`, replacing messages that share an *id*."""
    # Ensure every incoming message has an id
    # assign a unique id if missing
    for m in right:
        if not m.id:
            m.id = str(uuid4())

    # merge new messages with the existing messages
    merged = left.copy()
    for m in right:
        # replace existing message if it has the same id
        for i, existing in enumerate(merged):
            if existing.id == m.id:   # replace
                merged[i] = m
                break
        else:
            # append new messages to the end
            merged.append(m)          # append new
    return merged

In [3]:
# use reduce_messages to merge messages for the instead of simple add 
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

### 2.1 Tool – Tavily search (same as Lesson 4)

In [4]:
tool = TavilySearchResults(max_results=2)

### 3. Example of manual human approval  

In [5]:
class Agent:
    def __init__(self, model, tools, system: str = "", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")

        # NEW: pause **before** the action node runs
        self.graph = graph.compile(checkpointer=checkpointer,
                                   interrupt_before=["action"])

        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    # ----- Node callbacks -----
    def call_openai(self, state: AgentState):
        msgs = state["messages"]
        if self.system:
            msgs = [SystemMessage(content=self.system)] + msgs
        reply = self.model.invoke(msgs)
        return {"messages": [reply]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state["messages"][-1]
        # Decide whether the assistant wants to call a tool
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for call in tool_calls:
            print(f"🔧 Calling tool: {call}")
            res = self.tools[call["name"]].invoke(call["args"])
            results.append(ToolMessage(tool_call_id=call["id"],
                                       name=call["name"],
                                       content=str(res)))
        print("Back to the LLM")
        return {"messages": results}

### 4. Test the human-in-the-loop agent

In [6]:
prompt = """You are a smart research assistant.  
Use the search engine to look up information when necessary.  
Ask follow‑up questions if you’re unsure. """

model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

#### First conversation – notice the **pause** before tool calling

In [7]:
messages = [HumanMessage(content="What's the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)  # Stops **before** the action node

{'messages': [HumanMessage(content="What's the weather in SF?", id='b1485b73-872f-4f05-a23f-41b1cbafede2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_k05aoERS784ji0P9G1HjABQo', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 119, 'total_tokens': 142, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fdd6627a-ca99-4174-a759-4be83247a8bb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_k05aoERS784ji0P9G1HjABQo'}])]}
{'messages': [AIMessage(content='', addition

In [8]:
# Inspect the paused state
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content="What's the weather in SF?", id='b1485b73-872f-4f05-a23f-41b1cbafede2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'id': 'call_k05aoERS784ji0P9G1HjABQo', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 119, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 142}}, id='run-fdd6627a-ca99-4174-a759-4be83247a8bb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_k05aoERS784ji0P9G1HjABQo'}])]}, next=('action',), conf

In [9]:
# The graph tells us the next node is 'action' 
abot.graph.get_state(thread).next

('action',)

> Because of the interrupt, **no tool has run yet**. Let’s resume:

In [10]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

🔧 Calling tool: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_k05aoERS784ji0P9G1HjABQo'}
Back to the LLM
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1747052103, \'localtime\': \'2025-05-12 05:15\'}, \'current\': {\'last_updated_epoch\': 1747052100, \'last_updated\': \'2025-05-12 05:15\', \'temp_c\': 13.9, \'temp_f\': 57.0, \'is_day\': 0, \'condition\': {\'text\': \'Light rain\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/296.png\', \'code\': 1183}, \'wind_mph\': 11.6, \'wind_kph\': 18.7, \'wind_degree\': 189, \'wind_dir\': \'S\', \'pressure_mb\': 1013.0, \'pressure_in\': 29.92, \'precip_mm\': 1.5, \'precip_in\': 0.06, \'humidity\': 83, \'cloud\': 100, \'feelslike

Check the final state and verify `next` is empty:

In [11]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content="What's the weather in SF?", id='b1485b73-872f-4f05-a23f-41b1cbafede2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'id': 'call_k05aoERS784ji0P9G1HjABQo', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 119, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 142}}, id='run-fdd6627a-ca99-4174-a759-4be83247a8bb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_k05aoERS784ji0P9G1HjABQo'}]), ToolMessage(content='[{\

In [12]:
abot.graph.get_state(thread).next

()

#### Put everything together, loop for approval

In [13]:
messages = [HumanMessage(content="What's the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

# Interactive approval loop
while abot.graph.get_state(thread).next:
    proceed = input("Proceed with tool call? (y/n): ")
    if proceed.lower() != "y":
        print("Aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content="What's the weather in LA?", id='813b5471-e1bb-416a-b673-2668b094c42e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_S6Brcb9hLrgNdpRrH0431u6d', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 119, 'total_tokens': 142, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cf30be32-31b6-424f-83f4-ab59d1076aa4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}])]}
{'messages': [AIMessage(content='', additional_k

### 5. Exploring state memory & time‑travel

Run until the interrupt and then modify the state

In [14]:
# Capture a state snapshot
current_values = abot.graph.get_state(thread)
current_values

StateSnapshot(values={'messages': [HumanMessage(content="What's the weather in LA?", id='813b5471-e1bb-416a-b673-2668b094c42e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 119, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 142}}, id='run-cf30be32-31b6-424f-83f4-ab59d1076aa4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}])]}, next=('action',), config={

In [15]:
# Get the tool call
id_current_values = -1
current_values.values['messages'][id_current_values]

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 119, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 142}}, id='run-cf30be32-31b6-424f-83f4-ab59d1076aa4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}])

In [16]:
current_values.values['messages'][id_current_values].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Los Angeles'},
  'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}]

In [17]:
# Get the tool call ID
_id = current_values.values['messages'][id_current_values].tool_calls[0]['id']

# Replace the tool call with a new one (modify query)
current_values.values['messages'][id_current_values].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [18]:
# Update the state of the agent
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content="What's the weather in LA?", id='813b5471-e1bb-416a-b673-2668b094c42e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 119, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 142}}, id='run-cf30be32-31b6-424f-83f4-ab59d1076aa4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}])]}


{'configurable': {'thread_id': '2',
  'thread_ts': '1f02f2ad-e797-60a3-8002-06de6f1e24b4'}}

In [19]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content="What's the weather in LA?", id='813b5471-e1bb-416a-b673-2668b094c42e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 119, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 142}}, id='run-cf30be32-31b6-424f-83f4-ab59d1076aa4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}])]}, next=('action',), config={'c

In [20]:
# Stream the agent's responses
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

🔧 Calling tool: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}
Back to the LLM
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.4411, \'lon\': -91.0551, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1747052163, \'localtime\': \'2025-05-12 07:16\'}, \'current\': {\'last_updated_epoch\': 1747052100, \'last_updated\': \'2025-05-12 07:15\', \'temp_c\': 15.1, \'temp_f\': 59.2, \'is_day\': 1, \'condition\': {\'text\': \'Overcast\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/122.png\', \'code\': 1009}, \'wind_mph\': 3.1, \'wind_kph\': 5.0, \'wind_degree\': 70, \'wind_dir\': \'ENE\', \'pressure_mb\': 1017.0, \'pressure_in\': 30.02, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 73, \'cloud\': 0, \'feelslike_c\': 15.1, \'feel

In [21]:
# TIME TRAVEL !!!
# List the entire history (most recent first)
history = list(abot.graph.get_state_history(thread))
for snap in history:
    print(snap.values["count"] if "count" in snap.values else "–", snap)

– StateSnapshot(values={'messages': [HumanMessage(content="What's the weather in LA?", id='813b5471-e1bb-416a-b673-2668b094c42e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 119, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 142}}, id='run-cf30be32-31b6-424f-83f4-ab59d1076aa4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}]), ToolMessage(content='[{\'url

In [22]:
# Jump back three steps and resume from there
to_replay = history[-3]
for event in abot.graph.stream(None, to_replay.config):
    for v in event.values():
        print(v)

🔧 Calling tool: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}
Back to the LLM
{'messages': [ToolMessage(content='[{\'url\': \'https://weathershogun.com/weather/usa/ca/los-angeles/451/may/2025-05-12\', \'content\': \'Los Angeles, California Weather: Monday, May 12, 2025. Cloudy weather, overcast skies with clouds. Day 72°. Night 57°. Precipitation 0 %.\'}, {\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.0522, \'lon\': -118.2428, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1747052169, \'localtime\': \'2025-05-12 05:16\'}, \'current\': {\'last_updated_epoch\': 1747052100, \'last_updated\': \'2025-05-12 05:15\', \'temp_c\': 17.2, \'temp_f\': 63.0, \'is_day\': 0, \'condition\': {\'text\': \'Overcast\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/122.png

The new branch is stored **alongside** the original path, so you can compare or even merge results later.

#### Go back in time and redit

In [23]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content="What's the weather in LA?", id='813b5471-e1bb-416a-b673-2668b094c42e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 119, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 142}}, id='run-cf30be32-31b6-424f-83f4-ab59d1076aa4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}])]}


In [24]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

🔧 Calling tool: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}
Back to the LLM
{'messages': [ToolMessage(content="[{'url': 'https://www.accuweather.com/en/us/los-angeles-ca/90012/current-weather/37866_pc?lang=en-us&partner=web_stacksports_adc', 'content': 'Los Angeles, CA · Current Weather. 9:46 AM. 83°F. Mostly sunny. RealFeel® 89°. Very Warm. RealFeel Guide. Very Warm. 82° to 89°.'}, {'url': 'https://www.accuweather.com/en/us/los-angeles-ca/90012/current-weather/347625?lang=en-us&partner=web_scorebook_adc', 'content': 'Los Angeles, CA · Current Weather. 1:27 PM. 88°F. Sunny. RealFeel® 97°. Hot. RealFeel Guide. Hot. 90° to 100°. Caution advised.'}]", name='tavily_search_results_json', id='15d28739-afdb-4b8e-96f4-a0569a444a21', tool_call_id='call_S6Brcb9hLrgNdpRrH0431u6d')]}
{'messages': [HumanMessage(content="What's the weather in LA?", id='813b5471-e1bb-416a-b673-2668b094c42e'), AIMessage(content=

### Add message to a state at a given time

In [25]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']

# Update the state - added message for tool call action
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius",
)]}


In [26]:
branch_and_add = abot.graph.update_state(
    to_replay.config, 
    state_update, 
    as_node="action") # N.B. added as_node="action" to avoid tool call and return message directly 

In [27]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content="What's the weather in LA?", id='813b5471-e1bb-416a-b673-2668b094c42e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 119, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 142}}, id='run-cf30be32-31b6-424f-83f4-ab59d1076aa4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_S6Brcb9hLrgNdpRrH0431u6d'}]), ToolMessage(content='54 degree celcius', name='ta

> **Summary:** With `interrupt_before`, a human-in-the-loop can inspect/alter state *before* external actions occur. It is also possible to travel in time, modify states and return user defined messages as actions. 